### 실습목표

- 추천시스템의 개념과 목적을 이해한다.
- Implicit 라이브러리를 활용하여 Matrix - Factorization(이하 MF) 기반의 추천 모델을 만들어 본다.
- 음악 감상 기록을 활용하여 비슷한 아티스트를 찾고 아티스트를 추천해 본다.
- 추천 시스템에서 자주 사용되는 데이터 구조인 CSR Matrix을 익힌다
- 유저의 행위 데이터 중 Explicit data와 Implicit data의 차이점을 익힌다.
- 새로운 데이터셋으로 직접 추천 모델을 만들어 본다.

### 사전 고려 사항

- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
- 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해볼 수 있습니다.
- 별점을 시청횟수로 해석해서 생각하겠습니다.
- 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

## 데이터 준비와 전처리

In [1]:
import pandas as pd
import os
import numpy as np

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings_orig = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings_orig)
ratings_orig.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [3]:
# ratings = ratings_orig[ratings_orig['rating']>=3]
# 뒤에서 데이터를 수정할 때, 원본까지 수정할지, 복사본만 수정할지 몰라서 에러가 뜸
# copy()로 복사본임을 확실히 해주자

# 3점 이상만 남깁니다.
ratings = ratings_orig[ratings_orig['rating']>=3].copy()
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# ratings 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)
ratings.head()

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [5]:
# timstamp는 사용하지 않으므로 제거
# count 컬럼을 앞으로
using_cols = ['user_id', 'count', 'movie_id']
ratings = ratings[using_cols]
ratings.tail(10)

,user_id,count,movie_id
1000198,6040,3,2021
1000199,6040,5,2022
1000200,6040,5,2028
1000201,6040,4,1080
1000202,6040,4,1089
1000203,6040,3,1090
1000205,6040,5,1094
1000206,6040,5,562
1000207,6040,4,1096
1000208,6040,4,1097


## 분석

- ratings에 있는 유니크한 영화 개수
- rating에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)

In [6]:
# 영화 제목을 출력하기 위해 ratings과 movies를 movie_id 기준으로 합치기
# 따로 지정하지 않아도 똑같은 컬럼명이 있으면 그걸 기준으로 합쳐준다.
# 먼저 merge 한 후 favorite을 추가 했더니, 결측치 컬럼이 자꾸 생기길래 임시로 분리
temp = pd.merge(ratings, movies)
print(len(temp))
temp.tail()

836478


,user_id,count,movie_id,title,genre
836473,5851,5,3607,One Little Indian (1973),Comedy|Drama|Western
836474,5854,4,3026,Slaughterhouse (1987),Horror
836475,5854,3,690,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,4,2909,"Five Wives, Three Secretaries and Me (1998)",Documentary
836477,5948,5,1360,Identification of a Woman (Identificazione di ...,Drama


In [7]:
# 영화 개수 아이디와 제목 개수가 같은지 확인
print(temp['movie_id'].nunique())
print(temp['title'].nunique())

3628
3628


In [8]:
# 사용자 수
temp['user_id'].nunique()

6039

In [9]:
# 1부터 6040까지 있는데 왜때문에 6039명이죠..??
# 빈 번호 내꺼

print(set(range(1,6041)) - set(temp['user_id'].unique()))
my_id = list(set(range(1,6041)) - set(temp['user_id'].unique()))[0]

{3598}


In [10]:
# 인기 많은 영화
# title끼리 묶었을 때 user_id 갯수 세서 상위 30개 출력하기
# 별점의 총점수 같은 게 아니라 3점 이상의 별점을 매긴 사람의 수
artist_count = temp.groupby('title')['user_id'].count()
artist_count.sort_values(ascending=False).head(10)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Name: user_id, dtype: int64

## 내가 선호하는 영화 추가

In [11]:
# 영화 제목 찾기
movie_temp = movies.loc[movies['genre'].str.contains('Animation')].copy()
movie_temp['title'].unique()

array(['Toy Story (1995)', 'Balto (1995)', 'Pocahontas (1995)',
       'Goofy Movie, A (1995)', 'Gumby: The Movie (1995)',
       'Swan Princess, The (1994)', 'Lion King, The (1994)',
       'Pagemaster, The (1994)', 'Aladdin (1992)',
       'Snow White and the Seven Dwarfs (1937)',
       'Beauty and the Beast (1991)', 'Pinocchio (1940)',
       'Heavy Metal (1981)', 'Aristocats, The (1970)',
       'All Dogs Go to Heaven 2 (1996)',
       'James and the Giant Peach (1996)', 'Space Jam (1996)',
       'Oliver & Company (1988)',
       'Wallace & Gromit: The Best of Aardman Animation (1996)',
       'Ghost in the Shell (Kokaku kidotai) (1995)',
       'Close Shave, A (1995)', 'Hunchback of Notre Dame, The (1996)',
       'Land Before Time III: The Time of the Great Giving (1995)',
       'Cinderella (1950)', 'Winnie the Pooh and the Blustery Day (1968)',
       'Three Caballeros, The (1945)', 'Sword in the Stone, The (1963)',
       'Dumbo (1941)', "Pete's Dragon (1977)",
       'Alice

In [12]:
# 내가 좋아하는 영화 데이터 확인
my_favorite = ['Lion King, The (1994)',
               'Toy Story (1995)',
               'Toy Story 2 (1999)',
               'Wallace & Gromit: The Best of Aardman Animation (1996)',
               'Tarzan (1999)']
my_favorite_id = movies[movies['title'].isin(my_favorite)]
my_favorite_id

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
360,364,"Lion King, The (1994)",Animation|Children's|Musical
711,720,Wallace & Gromit: The Best of Aardman Animatio...,Animation
2618,2687,Tarzan (1999),Animation|Children's
3045,3114,Toy Story 2 (1999),Animation|Children's|Comedy


In [13]:
# my_id가 위 영화를 5회씩 봤다고 가정
my_playlist = pd.DataFrame({'user_id': [my_id]*5, 'count':['5']*5, 'movie_id': my_favorite_id['movie_id']})
# 3598  5     1 
# 3598  5   364
# 3598  5   720
# 3598  5  2687
# 3598  5  3114    마지막 행 이후에 추가됨

if not ratings.isin({'user_id':[int(my_id)]})['user_id'].any():  # user_id에 '3598'이라는 데이터가 없다면
    ratings = ratings.append(my_playlist)                    # 임의로 만든 my_favorite_id 데이터를 추가 

ratings.tail(10)       # 잘 추가되었는지 확인

,user_id,count,movie_id
1000203,6040,3,1090
1000205,6040,5,1094
1000206,6040,5,562
1000207,6040,4,1096
1000208,6040,4,1097
0,3598,5,1
360,3598,5,364
711,3598,5,720
2618,3598,5,2687
3045,3598,5,3114


In [14]:
# my_id 추가 전 후 데이터 수 비교
print(f'{len(ratings)} - {len(temp)} = {len(ratings) - len(temp)}')

836483 - 836478 = 5


## 인덱싱하기

주어진 데이터를 정수형태로 맞췄기 때문에 따로 인덱싱 작업이 필요 없을 거라고 생각했으나 

CSR matrix를 만드는 과정에서 계속 ValueError: row index exceeds matrix dimensions 에러가 났다

ratings.user_id 의 max가 ratings.user_id의 길이보다 작아야 한다길래 빈 번호를 찾아서 들어가봤으나

어차피 0번도 비어있어서 해결이 안되는 상황이었고

movie_id도 노답인 상황이라 다시 인덱싱 하기로 결정

처음엔 ratings만 이용해서 계산하고 movies를 합치려고 했는데

인덱싱 이후에 id가 바뀔테니 그냥 합쳐서 정리하기로 함

In [16]:
num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

print(ratings.user_id.unique().max(), ratings.movie_id.unique().max())
print(num_user, num_movie)

6040 3952
6040 3628


In [17]:
# ratings과 movies를 movie_id 기준으로 합치기
# 따로 지정하지 않아도 똑같은 컬럼명이 있으면 그걸 기준으로 합쳐준다.
# 먼저 merge 한 후 favorite을 추가 했더니, 결측치 컬럼이 자꾸 생기길래
# favororite 추가 후 merge
data = pd.merge(ratings, movies).copy()
data.tail()

,user_id,count,movie_id,title,genre
836478,5851,5,3607,One Little Indian (1973),Comedy|Drama|Western
836479,5854,4,3026,Slaughterhouse (1987),Horror
836480,5854,3,690,"Promise, The (Versprechen, Das) (1994)",Romance
836481,5938,4,2909,"Five Wives, Three Secretaries and Me (1998)",Documentary
836482,5948,5,1360,Identification of a Woman (Identificazione di ...,Drama


In [18]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = data['user_id'].unique()
movie_unique = data['title'].unique()

# 유저, 영화 indexing 하는 코드 
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

print('done')

done


In [19]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = data['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_movie_data
else:
    print('title column indexing Fail!!')

data.tail()

user_id column indexing OK!!
title column indexing OK!!


,user_id,count,movie_id,title,genre
836478,1621,5,3607,3623,Comedy|Drama|Western
836479,3481,4,3026,3624,Horror
836480,3481,3,690,3625,Romance
836481,4159,4,2909,3626,Documentary
836482,1648,5,1360,3627,Drama


In [20]:
num_user = data['user_id'].nunique()
num_movie = data['title'].nunique()

print(data.user_id.unique().max(), data.title.unique().max())
print(num_user, num_movie)

6039 3627
6040 3628


## CSR matrix 만들기

In [21]:
data['count']

0         5
1         5
2         4
3         4
4         5
         ..
836478    5
836479    4
836480    3
836481    4
836482    5
Name: count, Length: 836483, dtype: object

In [22]:
# count가 object 라서 매트릭스를 안 만들어주길래 자료형 바꿔줌

data['count'] = data['count'].astype(int)
data['count']

0         5
1         5
2         4
3         4
4         5
         ..
836478    5
836479    4
836480    3
836481    4
836482    5
Name: count, Length: 836483, dtype: int64

In [23]:
from scipy.sparse import csr_matrix

csr_data = csr_matrix((data['count'], (data.user_id, data.title)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 모델을 구성하여 훈련

als_model = AlternatingLeastSquares

1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지 
2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지 
3. use_gpu : GPU를 사용할 것인지 
4. iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지

1,4를 늘릴수록 학습데이터를 잘 학습하게 되지만 과적합의 우려가 있음

gpu 사용을 True 로 했더니

#GPU training requires factor size to be a multiple of 32. Increasing factors from 100 to 128.

라는 에러 메세지가 뜨길래 factor를 128로 바꿔줬는데 

ValueError: No CUDA extension has been built, can't train on GPU.

라는 메세지가 뜸.. 찾아봤더니 뭔가 conda 환경 문제인 것 같아서 그냥 False로 두기로 함

In [24]:
from implicit.als import AlternatingLeastSquares

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=128, 
                                    regularization=0.01, 
                                    use_gpu=False, 
                                    iterations=15, 
                                    dtype=np.float32)

# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [25]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

## 모델이 예측한 선호도 파악

내가 선호하는 5가지 영화 중 하나

title : Toy Story (1995), movie_id : 1

그 외의 영화 하나

title : Jumanji (1995), movie_id : 2

In [26]:
# my_id의 벡터
my_id_ = user_to_idx[my_id]
my_vector = als_model.user_factors[my_id_]
my_vector

array([-0.57822376, -0.39609516, -0.95414567,  0.666361  ,  0.2673493 ,
       -0.4501064 , -0.04945354,  0.1580482 ,  0.6322484 ,  0.3200209 ,
       -0.19412972,  0.26332662,  0.15475562, -0.24061759,  0.09403586,
        0.24886487,  0.05330142,  0.45221913, -0.24973969,  0.2833698 ,
       -0.2941154 , -0.20788637,  0.23111723,  0.553293  ,  0.49224874,
        0.13148674,  0.5210504 , -0.200784  ,  0.7350711 , -0.2556979 ,
        0.07121269, -0.4875192 , -0.57173586, -0.71051633, -0.14643447,
        0.17228894,  0.5749423 , -0.39857608,  0.94394374,  0.6344942 ,
       -0.31063595,  0.11416014, -0.23904975,  0.50572556,  0.00880369,
        0.09433257,  0.5521122 , -0.65214074, -0.62706244,  0.5224925 ,
        0.21000634,  0.12322996,  0.35617724, -0.07283975,  0.5146339 ,
       -0.49625942, -0.48951775,  0.39587075, -0.8576414 ,  0.38507512,
        0.5812303 ,  0.01762483,  0.06201517,  0.4319185 , -0.7842106 ,
        0.5418227 ,  0.22678448,  0.1645163 ,  0.8722017 ,  0.61

In [27]:
# ToyStory의 벡터
toy_story = movie_to_idx['Toy Story (1995)']
toy_story_vector = als_model.item_factors[toy_story]
toy_story_vector

array([ 0.00377748, -0.01356632, -0.0131848 ,  0.01707101,  0.01712736,
       -0.04330068,  0.00169006,  0.01165892,  0.00466082,  0.01764626,
        0.00047193,  0.00011556, -0.01049469,  0.01170299, -0.01412884,
        0.02554613,  0.03209743,  0.02154545, -0.00603585,  0.00351878,
        0.01710978, -0.0068976 ,  0.00620444, -0.00229199,  0.01840092,
        0.00220766,  0.00672136, -0.01769676,  0.03041925, -0.00121725,
       -0.01399142, -0.00626473,  0.0170452 , -0.02907768,  0.00740331,
        0.00440232,  0.02382636, -0.01571313,  0.02019827,  0.02448057,
       -0.01984956, -0.00841881,  0.00715517,  0.02655918, -0.01622684,
       -0.00613402,  0.02088703, -0.00967224, -0.02554182,  0.0250453 ,
        0.03715481, -0.00416041,  0.00777534,  0.00863113,  0.0231662 ,
       -0.01952973,  0.00628036,  0.03315172, -0.00085454, -0.0017914 ,
        0.02879839, -0.00840953,  0.02362895,  0.00517683, -0.02230731,
        0.0206351 ,  0.01276887,  0.01392343,  0.03818164,  0.01

In [28]:
# Jumanji의 벡터
jumanji = movie_to_idx['Jumanji (1995)']
jumanji_vector = als_model.item_factors[jumanji]
jumanji_vector

array([ 1.40408520e-02,  2.25273557e-02, -1.17816307e-04, -2.47508544e-03,
       -6.61492534e-03, -2.59571330e-04, -7.86825840e-04,  1.25556476e-02,
       -8.58507876e-04,  1.93150882e-02, -1.37526996e-03, -2.01954274e-03,
        8.08058400e-03,  2.04543024e-02,  2.85883211e-02,  1.14415847e-02,
        1.30861131e-02,  3.30699841e-03,  1.82637974e-04,  2.01718110e-04,
        7.69391237e-03,  3.67505848e-02, -8.03311239e-04,  3.69293839e-02,
        4.04995307e-03,  1.94324031e-02,  2.12151688e-02,  1.74061544e-02,
       -2.24698131e-04,  6.55654585e-03, -2.39027943e-03, -1.69055432e-03,
        1.60440411e-02, -1.95760489e-03,  2.32521421e-03, -1.27604110e-02,
        3.25213396e-03,  5.33046946e-03,  2.16725655e-02, -1.24639757e-02,
       -1.77033357e-02, -5.43592684e-03, -6.47336070e-04,  2.22310461e-02,
        1.00413021e-02,  7.09674368e-03,  6.60301885e-04,  1.73738468e-02,
       -6.05892274e-04, -7.31942337e-03, -3.00682476e-03,  7.51550961e-03,
        3.70097496e-02,  

In [29]:
# my_vector와 toy_story_vector를 내적
np.dot(my_vector, toy_story_vector)

0.74432373

In [30]:
# my_vector와 jumanji_vector를 내적
np.dot(my_vector, jumanji_vector)

0.08010477

## 내가 좋아하는 영화와 비슷한 영화

In [43]:
# movie_to_idx 를 뒤집어, index로부터 movie의 제목을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}

# AlternatingLeastSquares 클래스에 구현되어 있는
# similar_items 메서드를 통하여 비슷한 영화 찾는 함수 만들기
def get_similar_movie(movie_title: str):
    title_id = movie_to_idx[movie_title]
    similar_movie = als_model.similar_items(title_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

get_similar_movie('Wallace & Gromit: The Best of Aardman Animation (1996)')

['Wallace & Gromit: The Best of Aardman Animation (1996)',
 'Wrong Trousers, The (1993)',
 'Grand Day Out, A (1992)',
 'Close Shave, A (1995)',
 'Creature Comforts (1990)',
 'Batman: Mask of the Phantasm (1993)',
 'I Married A Strange Person (1997)',
 'Hype! (1996)',
 'Kid, The (1921)',
 'Better Living Through Circuitry (1999)']

In [37]:
get_similar_movie('Star Wars: Episode IV - A New Hope (1977)')

['Star Wars: Episode IV - A New Hope (1977)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Raiders of the Lost Ark (1981)',
 'Star Wars: Episode I - The Phantom Menace (1999)',
 'Alien (1979)',
 'E.T. the Extra-Terrestrial (1982)',
 'Back to the Future (1985)',
 'Matrix, The (1999)',
 'Terminator, The (1984)']

## 내가 좋아할 만한 영화

In [44]:
# AlternatingLeastSquares 클래스의 recommend 메서드를 이용해 영화 추천 받는 함수
# recommend에서는 user*item CSR Matrix를 받습니다.
# filter_already_liked_items 는 유저가 이미 평가한 아이템은 제외하는 Argument

def get_recommend_movie(my_id: int):
    user = user_to_idx[my_id]
    movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
    movie_recommended = [idx_to_movie[i[0]] for i in movie_recommended]
    return movie_recommended

get_recommend_movie(my_id)

["Bug's Life, A (1998)",
 'Aladdin (1992)',
 'Beauty and the Beast (1991)',
 'Wrong Trousers, The (1993)',
 'Iron Giant, The (1999)',
 'Mulan (1998)',
 'Close Shave, A (1995)',
 'Antz (1998)',
 'Grand Day Out, A (1992)',
 'Hercules (1997)',
 'Babe (1995)',
 'James and the Giant Peach (1996)',
 'Prince of Egypt, The (1998)',
 'Hunchback of Notre Dame, The (1996)',
 'Chicken Run (2000)',
 'Creature Comforts (1990)',
 'Little Mermaid, The (1989)',
 'Sixth Sense, The (1999)',
 'South Park: Bigger, Longer and Uncut (1999)',
 'Jungle Book, The (1967)']

In [45]:
# 추천한 콘텐츠의 점수(합이 콘텐츠의 점수)에 기여한
# 다른 콘텐츠와 기여도를 반환하는 메서드를 이용한 함수

def get_contribution(movie_title: str):
    title_id = movie_to_idx[movie_title]
    user = user_to_idx[my_id]
    explain = als_model.explain(user, csr_data, itemid=title_id)
    explain = [(idx_to_movie[i[0]], i[1]) for i in explain[1]]
    return explain

get_contribution('Chicken Run (2000)')

[('Wallace & Gromit: The Best of Aardman Animation (1996)',
  0.09529473067768784),
 ('Toy Story 2 (1999)', 0.08929240281135135),
 ('Toy Story (1995)', 0.030313683624765653),
 ('Lion King, The (1994)', 0.0028787519301335046),
 ('Tarzan (1999)', -1.592658294689289e-05)]

In [47]:
get_contribution("Bug's Life, A (1998)")

[('Toy Story 2 (1999)', 0.24256089547896195),
 ('Toy Story (1995)', 0.16263144064926818),
 ('Tarzan (1999)', 0.08167275393151),
 ('Wallace & Gromit: The Best of Aardman Animation (1996)',
  0.014546435305833365),
 ('Lion King, The (1994)', 0.004572137259285657)]

# 회고

#### 첫번째 시도

ratings와 movies를 합친 후 favorite을 추가했더니, favorite의 내용을 채우기 위해 다시 merge 할때 제대로 병합이 안됐다.

영화 제목 출력 시에만 임시로 병합하기로 결정

#### 두번째 시도

어차피 유저데이터와 영화 데이터만 필요하므로 ratings와 movie를 병합하지 않고

인덱싱 없이 정수 데이터인 user_id와 movie_id 만으로 CSR Matrix를 만들려고 시도했다.

row가 뭘 벗어났다고 계속 오류가 떠서 확인해 봤더니

데이터데 있는 최대값이, 총 갯수보다 적어야만 제대로 만들어질 수 있었다.

user_id는 0번과 3598번이 비어있고, movie_id도 빈 번호들이 너무 많아서

그냥 다시 인덱싱을 하기로 결정

#### 세번째 시도 

movie_id를 인덱싱하면, 영화 제목 출력을 위해 ratings와 movie를 merge 할때

번호가 어긋날 거라서, 먼저 병합한 후 인덱싱을 했다

그랬더니 출력 결과물을 movie_id로 뱉어냄....

나온 결과물을 다시 title로 전환 하려면 할 수도 있겠지만

그냥 애초에 title을 인덱싱하는게 깔끔하다고 생각되어 마지막 수정

### 아쉬운 점

뭘 조절해서 결과물이 잘 나왔는지 안 나왔는지 확인하고 싶은데

아는게 있어야 확인을 하지....

지금 상태로도 몇개 비슷해보이는 유형의 영화가 보이기는 하는데

애초에 애니메이션 장르가 좀 특이한 편이라서 그럭저럭 잘 나온 것 같기도 하다

데이터를 다루는데 도메인이 얼마나 중요한지 새삼 느끼게 되는 노드였다.ㅠㅠ